In [1]:
%pylab
%run LoadNetworkArgs.py 64 1 1  
# arguments are: Network size (# Nodes, 64, 128 or 256), Network index (integer for different networks), Random seed (integer)

import warnings
warnings.filterwarnings("ignore")

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


QStandardPaths: wrong ownership on runtime directory /mnt/wslg/runtime-dir, 1000 instead of 0


# Set sources and targets

In [2]:
# set input and output
sources = 10    # number of source nodes
targets = 3     # number of target nodes

# select nodes for inputs and outputs
RNodes = np.random.choice(arange(NN-1), size=sources+targets, replace=False)  # Source and target nodes together
KNodes = list(set(arange(NN)) - set(RNodes))
SourceNodes = RNodes[:sources]     # Source node indices
TargetNodes = RNodes[sources:]     # Target node indices
GroundNode = [NN-1]                # Ground nodes is the node in the last index

Data = randn(1,sources)               # Source values
Comp = randn(targets) * sum(Data)/5.  # Target values


# Train network

In [3]:
# Set all conductance values to 1
K = ones(NE)

# Compute all pressures given inputs and resistance values
P = GetPs(Data, SourceNodes, K)

# Compute all pressure drops over edges
DP = DM.dot(P)


In [4]:
Kticks = 512                                        # Number of discrete conductance values
KVals = linspace(2./Kticks,2.,Kticks)               # Discrete, equally spaced conductance values between 2./Kticks and 2.
dkv = KVals[1]-KVals[0]                             # Discrete spacing value
KInds = ones(NE,dtype=int) * (Kticks//2 - 1)        # Set all conductance values to the middle of ths scale K=1

In [5]:
Steps = 401        # Number of iterations
eta = 1.e-3        # Nudge parameter for clamped state
printEvery = 10    # Set number of steps between prints

KFInds = ones(NE,dtype=int) * (Kticks//2 - 1)      # Set all free conductance values to the middle of ths scale K=1
KCInds = ones(NE,dtype=int) * (Kticks//2 - 1)      # Set all clamped conductance values to the middle of ths scale K=1
KF = KVals[KFInds]                                 # Free network conductance values for current iteration
KC = KVals[KCInds]                                 # Free network conductance values for current iteration

Costs = zeros(Steps)
# Compute cost function at time t=0
CEq = CostSingleK(Data, SourceNodes, TargetNodes, KF, Comp)
Costs[0] = CEq

print(0, CEq)

for steps in range(1,Steps):
    # exp coupled learning
    KF = KVals[KFInds]   # Free conductance values for current iteration
    KC = KVals[KCInds]   # Clamped conductance values for current iteration

    # Free state computation
    PF = GetPs(Data, SourceNodes, KF)
    DPF = DM.dot(PF)
    PPF = DPF**2.   # Squared pressure drops
    #PPF = abs(DPF)

    # Compute the target values for the free state
    FreeTargets = PF[TargetNodes]
    
    # Define the nudge for the clamped state
    Nudge = (1. - eta) * FreeTargets + eta * Comp

    # Clamped state computations
    InpNudge = r_[Data[0], Nudge]    # inputs and nudge values together
    PC = GetPs(InpNudge, RNodes, KC)
    DPC = DM.dot(PC)
    PPC = DPC**2.   # Squared pressure drops
    #PPC = abs(DPC)

    # Coupled learning rule
    DKE = + PPC - PPF
    # Discretized rule according to signs only, change the index vector for each edge by +- 1
    DI = array(sign(DKE),dtype=int)
    KFInds = KFInds - DI
    KFInds = KFInds.clip(0,Kticks-1)   # Avoid clipping beyond the range of conductance values
    KCInds = KCInds - DI
    KCInds = KCInds.clip(0,Kticks-1)   # Avoid clipping beyond the range of conductance values
    NK = KVals[KFInds]                 # New conductance values
    CDK = norm(NK - KF)                # Magnitude of change in conductance
    KF = NK.copy()                     # Set new conductance values

    CEq = CostSingleK(Data, SourceNodes, TargetNodes, KF, Comp)
    Costs[steps] = CEq


    if steps%printEvery == 0:
        CEq = CostSingleK(Data, SourceNodes, TargetNodes, KF, Comp)
        print(steps, CEq)




0 0.5837352030284657
10 0.5438261269640272
20 0.5051374811026403
30 0.4677825791110701
40 0.43155580634422047
50 0.3965256879043387
60 0.362855534589569
70 0.3306378096043885
80 0.29996755683342363
90 0.2709400447988496
100 0.2436351281394774
110 0.21807474015525463
120 0.1941241170302294
130 0.1717490212430078
140 0.1504646315296153
150 0.13036274843505802
160 0.1111586665632893
170 0.09234342588741293
180 0.07407321970661232
190 0.05631521621910136
200 0.03983213188696904
210 0.025851871907289686
220 0.015202522612751838
230 0.007184687679036353
240 0.0019795769718328714
250 2.8207146756931497e-05
260 2.6974763192150027e-05
270 2.6703835368593474e-05
280 2.6457922379008856e-05
290 2.621086976927988e-05
300 2.5963681000263286e-05
310 2.571585524425666e-05
320 2.5466926517651812e-05
330 2.5217151774812142e-05
340 2.5085461757001794e-05
350 2.498218979729089e-05
360 2.4877576658202017e-05
370 2.4790822131317236e-05
380 2.470691224372895e-05
390 2.462091045419797e-05
400 2.45329553669584

In [6]:
figure(figsize=(8,6))   # Define figure
T = arange(Steps)       # Time axis
plot(T, Costs, lw=2)

xlabel('Steps', size=16)
ylabel('Error', size=16)
xticks(size=16)
yticks(size=16)

yscale('log')

# Draw network

In [43]:
import networkx as nx

Edges = c_[EI, EJ]
G = nx.from_edgelist(Edges)

NodeCols = array(['#000000']*NN)
NodeCols[SourceNodes] = '#d62728'
NodeCols[TargetNodes] = '#1f77b4'
NodeCols[GroundNode] = '#7f7f7f'

NodeSizes = array([100]*NN)
NodeSizes[SourceNodes] = 200
NodeSizes[TargetNodes] = 200
NodeSizes[GroundNode] = 200

# draw network with edge widths proportional to conductance values
figure(figsize=(8,6))
nx.draw(G, pos=nx.spring_layout(G, seed=100), width=KF, node_size=NodeSizes, node_color=NodeCols)
